In [1]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
cd /opt/ml/code/_mmdetection_trash

/opt/ml/code/_mmdetection_trash


In [3]:
# !wget http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.load_from = 'faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'

PREFIX = '../../input/data/'

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = (512, 512)

cfg.seed=2020
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'
cfg.runner.max_epochs = 15

cfg.model.roi_head.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='FasterRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch'),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.0),
        loss_bbox=dict(typ

In [5]:
import pickle
with open('config_standard', 'wb') as f:
    pickle.dump(cfg, f)

In [6]:
model = build_detector(cfg.model)

2021-05-20 06:09:37,014 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-20 06:09:37,015 - mmdet - INFO - Use load_from_torchvision loader
2021-05-20 06:09:37,269 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [7]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.77s)
creating index...
index created!


In [8]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-20 06:09:45,065 - mmdet - INFO - load checkpoint from faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2021-05-20 06:09:45,065 - mmdet - INFO - Use load_from_local loader
2021-05-20 06:09:45,215 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([44, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([44]).
2021-05-20 06:09:45,220 - mmdet - INFO - Start running, 

Done (t=0.86s)
creating index...
index created!


2021-05-20 06:10:15,996 - mmdet - INFO - Epoch [1][50/328]	lr: 1.978e-03, eta: 0:49:57, time: 0.615, data_time: 0.060, memory: 3695, loss_rpn_cls: 0.1185, loss_rpn_bbox: 0.0453, loss_cls: 0.9191, acc: 80.1392, loss_bbox: 0.4732, loss: 1.5560, grad_norm: 5.3404
2021-05-20 06:10:43,403 - mmdet - INFO - Epoch [1][100/328]	lr: 3.976e-03, eta: 0:46:44, time: 0.548, data_time: 0.011, memory: 3695, loss_rpn_cls: 0.0796, loss_rpn_bbox: 0.0492, loss_cls: 0.4418, acc: 88.4570, loss_bbox: 0.4281, loss: 0.9986, grad_norm: 2.3327
2021-05-20 06:11:11,084 - mmdet - INFO - Epoch [1][150/328]	lr: 5.974e-03, eta: 0:45:30, time: 0.554, data_time: 0.012, memory: 3695, loss_rpn_cls: 0.0861, loss_rpn_bbox: 0.0538, loss_cls: 0.3986, acc: 89.0068, loss_bbox: 0.3463, loss: 0.8848, grad_norm: 2.7674
2021-05-20 06:11:38,582 - mmdet - INFO - Epoch [1][200/328]	lr: 7.972e-03, eta: 0:44:35, time: 0.550, data_time: 0.012, memory: 3695, loss_rpn_cls: 0.0748, loss_rpn_bbox: 0.0498, loss_cls: 0.3622, acc: 89.7974, loss

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 22.0 task/s, elapsed: 30s, ETA:     0s

...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 21.2 task/s, elapsed: 31s, ETA:     0s

2021-05-20 07:06:44,714 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.53s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.91s).
Accumulating evaluation results...


2021-05-20 07:06:50,964 - mmdet - INFO - Epoch(val) [15][328]	bbox_mAP: 0.2330, bbox_mAP_50: 0.3760, bbox_mAP_75: 0.2480, bbox_mAP_s: 0.0440, bbox_mAP_m: 0.2140, bbox_mAP_l: 0.4080, bbox_mAP_copypaste: 0.233 0.376 0.248 0.044 0.214 0.408


DONE (t=0.75s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.233
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.376
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.248
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.044
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.214
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.408
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.376
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.118
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.352
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.640
